1. (obsolete) Specify all parameters to be varied?


# 2. Specify CAD model to be printed (.stl)

In [ ]:
# stl_path = '/Users/norapusok/models/flk.stl'
stl_path = '/Users/norapusok/slicer/models/20mm_cube.stl'

In [ ]:
import numpy
from stl import mesh

# Using an existing stl file:
your_mesh = mesh.Mesh.from_file(stl_path)

### Visualize imported mesh

In [ ]:
#  https://pypi.org/project/numpy-stl/
from mpl_toolkits import mplot3d
from matplotlib import pyplot

def plot_my_stl(stl_path):
    # Create a new plot
    figure = pyplot.figure()
    axes = figure.add_subplot(projection='3d')

    # Load the STL files and add the vectors to the plot
    your_mesh = mesh.Mesh.from_file(stl_path)
    axes.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

    # Auto scale to the mesh size
    #scale = your_mesh.points.flatten()
    #axes.auto_scale_xyz(scale, scale, scale)

    # Show the plot to the screen
    pyplot.show()

# 2a. Or create parametric geometry and turn it into a mesh
### modify side

In [ ]:
#brew install --cask freecad
import sys
sys.path.append('/Applications/FreeCAD.app/Contents/Resources/lib') 
import FreeCAD, Part #import FreeCADGui

# pip install PySide2
import Mesh
import Draft

In [ ]:
# FreeCAD create cube    
def makeBox(side = '20 mm'):
    App.newDocument()
    App.ActiveDocument.addObject("Part::Box","Box")
    App.ActiveDocument.ActiveObject.Label = "Cube"
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Width = side
    FreeCAD.ActiveDocument.getObject('Box').Length = side
    FreeCAD.ActiveDocument.getObject('Box').Height = side
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Shape.exportStl('box.stl')
# export as stl https://gist.github.com/hyOzd/2e75a9816cfabeb5b4aa

# Add label
def makeLabel(label_string="L0"):
    # https://wiki.freecad.org/Draft_ShapeString_tutorial
    ss=Draft.make_shapestring(String=label_string, FontFile="/Library/Fonts/DejaVuSans.ttf", Size=5.0, Tracking=0.0)
    plm=FreeCAD.Placement()
    plm.Base=FreeCAD.Vector(4.0, 4.0, side=20)
    plm.Rotation.Q=(0.0, 0.0, 0, 1.0)
    ss.Placement=plm
    ss.Support=None
    Draft.autogroup(ss)
    FreeCAD.ActiveDocument.recompute()
    
    App.ActiveDocument.addObject('Part::Extrusion','Extrude')
    f = App.ActiveDocument.getObject('Extrude')
    f.Base = App.ActiveDocument.getObject('ShapeString')
    f.DirMode = "Normal"
    f.DirLink = None
    f.LengthFwd = 5.000000000000000
    f.LengthRev = 0.000000000000000
    f.Solid = False
    f.Reversed = False
    f.Symmetric = False
    f.TaperAngle = 0.000000000000000
    f.TaperAngleRev = 0.000000000000000
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('ShapeString').Visibility = False
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('Extrude').Placement = App.Placement(App.Vector(2,2,0),App.Rotation(App.Vector(0,0,1),0))
    App.ActiveDocument.recompute()
    #FreeCAD.ActiveDocument.getObject('Extrude').Shape.exportStl('ext.stl')
    __objs__ = []
    __objs__.append(FreeCAD.ActiveDocument.getObject("Box"))
    __objs__.append(FreeCAD.ActiveDocument.getObject("Extrude"))
    Mesh.export(__objs__, u"/Users/norapusok/slicer/" + "comb_" + label_string + ".stl")


#### For now add these handwritten labels

In [ ]:
makeBox()
for l in ["L0", "L1", "L2", "L3"]:
    makeLabel(label_string=l)
# or stl merge https://pypi.org/project/numpy-stl/

#### Numpy alternative

In [ ]:
import numpy as np
from stl import mesh

# Define the 8 vertices of the cube
vertices = np.array([\
    [-1, -1, -1],
    [+1, -1, -1],
    [+1, +1, -1],
    [-1, +1, -1],
    [-1, -1, +1],
    [+1, -1, +1],
    [+1, +1, +1],
    [-1, +1, +1]])
# Define the 12 triangles composing the cube
faces = np.array([\
    [0,3,1],
    [1,3,2],
    [0,4,7],
    [0,7,3],
    [4,5,6],
    [4,6,7],
    [5,1,2],
    [5,2,6],
    [2,3,6],
    [3,7,6],
    [0,1,5],
    [0,5,4]])

# Create the mesh
cube = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        cube.vectors[i][j] = vertices[f[j],:]

# Write the mesh to file "cube.stl"
cube.save('cube.stl')
plot_my_stl('cube.stl')

### View combi.stl in Prusa

# 3. Slicing 

In [ ]:
# https://github.com/valkyriesavage/objectify
import subprocess

def slice_mesh(stl_location, h, pat):
    slice_command = ['/Applications/PrusaSlicer.app/Contents/MacOS/PrusaSlicer',
                    '--load', 'config.ini', 
                    '--layer-height', h,
                    '--fill-pattern', pat, 
                    '--export-gcode', stl_location]
    results = subprocess.check_output(slice_command)
    
    # the last line from Prusa Slicer is "Slicing result exported to ..."
    last_line = results.decode('utf-8').strip().split("\n")[-1]
    location = last_line.split(" exported to ")[1]

    return location

In [ ]:
# https://github.com/bambulab/BambuStudio/wiki/Command-Line-Usage

def slice_bambu(stl_path):
    slice_command = ['/Applications/BambuStudio.app/Contents/MacOS/BambuStudio'
                    , '--debug 2' 
                    #, '--orient'
                    #, '--arrange 1'
                    , '--load-settings "machine.json;process.json"'
                    #, '--load-filaments filament.json'
                    #, '--slice 0'  # slice all plates (spec. plate j for 3mf only!)
                    #, '--export-3mf'
                    #, 'output.3mf'
                    , stl_path]
    results = subprocess.check_output(slice_command)
    return results

In [ ]:
path = u"/Users/norapusok/slicer/models/"
paths = ["L0.stl", "L1.stl", "L2.stl", "L3.stl"]
heights = ['0.01', '0.4'] # nozzle 0.4
top_patterns = ['triangles', 'concentric']
cs = []
i = 0

for h in heights:
    for p in top_patterns:
        stl = path + paths[i]
        gcode_path = slice_mesh(stl, h, p)
        i+=1
        print(gcode_path) 
        
# /Applications/PrusaSlicer.app/Contents/MacOS/PrusaSlicer --help-fff

### View L0/L3.gcode in Prusa

# 4. Go print gcode files manually. 


In [ ]:
# or send to simplyprint.io for later

# 5. Take measurements using a caliper. Show .csv

# 6. NHST null hypothesis significance testing 


### Automatically select and execute statistical test using TEA

In [ ]:
import subprocess
# pip install tealang
import tea

##### 1/5 Dataset (rows are treated as individual observations, i.e. all variables are between-subjects.)

In [ ]:
data_path = "./cube_observations.csv"

##### 2/5 Variables 

#### Data types short help
##### nominal (discrete categories)
##### ordinal (discrete ordered categories)
##### numeric (continuous values!, range spec. fx [0,1] is optional)
##### - interval scale: real-additive (+)&(-) only valid ops.; 0-only a value (does not indicate lack of value); fx temperature/12H clock
##### - ratio scale: real-multiplicative (*)&(/)&(mean/median) also valid. ops.; 0 spec. elem fx. object's physical measurements 
      think binomial vs count (discrete)


In [ ]:

variables = [
    {
        'name' : 'layer height',
        'data type' : 'ratio'
        # ,'range' : [0.08, 0.28] #optional par.
    },
    {
        'name' : 'material',
        'data type' : 'nominal',
        'categories' : ['PLA', 'PETG', 'ABS']    # , 'nylon', 'metal']
    },
    {
        'name' : 'shrinkage x-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'shrinkage y-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'shrinkage z-axis',
        'data type' : 'ratio'
    }
        
# other variables infill pattern, travel speed?
]

##### 3/5 Study design (describe independent variables and dependent variables. multiple hypothesis?)

In [ ]:
study_design = {
    'study type': 'experiment',
    'independent variables': ['layer height', 'material'],
    'dependent variables': ['shrinkage x-axis', 'shrinkage y-axis', 'shrinkage z-axis']
}

##### 4/5 (optional )Assumptions (explicitely provide domain knowledge - relevant to testing your hypothesis)

In [ ]:
# physical phenomena usually normally distributed. 
#assumptions = {
#    'normal distribution': ['shrinkage x-axis', 'shrinkage y-axis', 'shrinkage z-axis']
#}
#tea.assume(assumptions, 'strict')   # mode relaxed(enforce user's choice) / strict(override)

##### 5/5 Hypothesis

In [ ]:
# example usage 
tea.data(data_path) # no "key" column provided, so between subject study.  # (data_path, key='ID')
                    # Exactly one observation per print parameter settings
tea.define_variables(variables)
tea.define_study_design(study_design)
tea.hypothesize(['material', 'shrinkage z-axis' ], ['material:PETG > ABS'])

In [ ]:
"""
Results:
--------------
Test: kruskall_wallis
***Test assumptions:
Independent (not paired) observations: material
Exactly one explanatory variable: material
Exactly one explained variable: shrinkage x-axis
Continuous (not categorical) data: shrinkage x-axis
Variable is categorical: material
Variable has two or more categories: material

***Test results:
name = Kruskall Wallis
test_statistic = 9.25461
p_value = 0.00978
adjusted_p_value = 0.00978
alpha = 0.05
dof = 4
Null hypothesis = There is no difference in medians between material = PLA, PETG, ABS on shrinkage x-axis.
Interpretation = t(4) = 9.25461, p = 0.00978. Reject the null hypothesis at alpha = 0.05. There is a difference in medians of shrinkage x-axis for at least one of material = PLA, PETG, ABS.

"""